In [ ]:
import fitz  # PyMuPDF
import re
from openai import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
import json # LLM 응답을 JSON으로 파싱하기 위함
api_key = os.getenv("OPENAI_API_KEY")
# OpenAI 클라이언트 초기화
# API 키는 환경 변수에서 자동 로드됨.
# 또는 client = OpenAI(api_key="YOUR_API_KEY") 로 직접 설정
client = OpenAI(api_key)

# 1. PDF 텍스트 추출 (페이지 번호 포함)
def extract_text_with_page_info(pdf_path):
    """
    PDF 파일에서 텍스트를 페이지별로 추출하고, 각 페이지 시작에 페이지 번호를 표기합니다.
    """
    document = fitz.open(pdf_path)
    full_text_with_pages = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()
        # 각 페이지 시작에 페이지 번호를 명시적으로 추가
        full_text_with_pages.append(f"\n---PAGE_START_{page_num+1}---\n{text}")
    return "\n".join(full_text_with_pages)

# 2. 초기 텍스트 분할 (LLM 컨텍스트에 맞게)
def initial_text_split(text, chunk_size=4000, chunk_overlap=200):
    """
    텍스트를 LLM이 처리할 수 있는 크기로 초기 분할합니다.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        # 분할 우선순위: 큰 의미 단위 -> 작은 의미 단위
        separators=["\n---PAGE_START_\d+---\n", "\n\n", "\n", ". ", "? ", "! ", " ", ""]
    )
    docs = text_splitter.create_documents([text])
    return docs

# 3. LLM을 이용한 의미적 청킹 (요구사항 추출 예시)
def llm_based_semantic_chunking(initial_chunks, llm_model="gpt-4o"):
    """
    각 초기 청크를 LLM에 전달하여 의미 있는 작은 청크 (예: 개별 요구사항)를 추출합니다.
    """
    final_semantic_chunks = []

    system_prompt = """
    당신은 RFP(제안요청서) 문서에서 핵심 요구사항을 정확하게 추출하고 구조화하는 전문 AI입니다.
    사용자가 제공하는 텍스트는 RFP 문서의 일부입니다.
    텍스트에서 모든 기능적(Functional) 및 비기능적(Non-Functional) 요구사항을 식별하여 추출해야 합니다.
    각 요구사항은 하나의 독립적인 의미 단위를 구성해야 하며, 간결하고 명확하게 요약되어야 합니다.
    각 요구사항의 출처 페이지를 반드시 명시하세요. (예: [페이지 N])
    
    응답은 다음 JSON 배열 형식으로만 제공해야 합니다.
    [
        {{
            "id": "REQ-001",
            "type": "기능적" or "비기능적" or "제약사항" or "기타",
            "description": "요구사항의 간결한 설명",
            "source_pages": [페이지 번호1, 페이지 번호2],
            "raw_text_snippet": "해당 요구사항이 포함된 원본 텍스트의 일부"
        }},
        ...
    ]
    만약 요구사항이 없다면 빈 배열 `[]`을 반환하세요.
    """

    for i, chunk_doc in enumerate(initial_chunks):
        chunk_text = chunk_doc.page_content
        print(f"--- LLM 처리 중: 청크 {i+1}/{len(initial_chunks)} (길이: {len(chunk_text)}자) ---")

        # 초기 청크에서 페이지 번호 범위 추출
        # 첫 번째 페이지 시작 표시에서 페이지 번호 추출 (fallback)
        match = re.search(r'---PAGE_START_(\d+)---', chunk_text)
        start_page_in_chunk = int(match.group(1)) if match else 1

        # 청크 내의 모든 페이지 시작 표시를 찾아 페이지 범위 지정
        page_numbers_in_chunk = sorted(list(set(
            int(p) for p in re.findall(r'---PAGE_START_(\d+)---', chunk_text)
        )))
        
        # 청크가 여러 페이지에 걸쳐있을 경우, 범위로 표현
        if page_numbers_in_chunk:
            # 청크 내 페이지 번호들을 실제 페이지 번호로 사용
            current_pages_str = f"페이지 범위: {page_numbers_in_chunk[0]} - {page_numbers_in_chunk[-1]}"
        else:
            current_pages_str = f"추정 페이지: {start_page_in_chunk}"

        user_prompt = f"""
        다음은 RFP 문서의 일부입니다. 이 부분에서 모든 요구사항을 JSON 형식으로 추출해 주세요.
        현재 처리 중인 텍스트의 추정 페이지 범위는 {current_pages_str} 입니다.

        --- 텍스트 ---
        {chunk_text}
        """

        try:
            response = client.chat.completions.create(
                model=llm_model,
                response_format={"type": "json_object"}, # JSON 응답을 강제
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ],
                temperature=0.0 # 창의성 최소화, 정확한 추출 지향
            )
            
            # JSON 응답 파싱
            llm_response_content = response.choices[0].message.content
            extracted_data = json.loads(llm_response_content)
            
            if isinstance(extracted_data, list):
                for req in extracted_data:
                    # 'id'가 없으면 생성 (REQ-XXX 형태)
                    if 'id' not in req:
                        req['id'] = f"REQ-{len(final_semantic_chunks) + 1:03d}"
                    # source_pages가 없으면 현재 청크의 페이지 번호 추가
                    if 'source_pages' not in req or not req['source_pages']:
                        req['source_pages'] = page_numbers_in_chunk if page_numbers_in_chunk else [start_page_in_chunk]
                    final_semantic_chunks.append(req)
            elif isinstance(extracted_data, dict) and 'requirements' in extracted_data and isinstance(extracted_data['requirements'], list):
                # { "requirements": [...] } 형태의 응답도 처리
                for req in extracted_data['requirements']:
                    if 'id' not in req:
                        req['id'] = f"REQ-{len(final_semantic_chunks) + 1:03d}"
                    if 'source_pages' not in req or not req['source_pages']:
                        req['source_pages'] = page_numbers_in_chunk if page_numbers_in_chunk else [start_page_in_chunk]
                    final_semantic_chunks.append(req)
            else:
                print(f"경고: 예상치 못한 JSON 형식 응답. {llm_response_content[:100]}...")

        except json.JSONDecodeError as e:
            print(f"JSON 파싱 오류: {e}. 응답: {llm_response_content[:200]}...")
        except Exception as e:
            print(f"LLM API 호출 중 오류 발생: {e}")
            
    return final_semantic_chunks

# --- 사용 예시 ---
if __name__ == "__main__":
    pdf_file_path = "./data/8. (제안요청서) 요구사항 분석 자동화_AI ITS 혁신팀.pdf"
    
    # 1. 전체 텍스트 추출 (페이지 정보 포함)
    print("PDF 텍스트 추출 중...")
    full_document_text = extract_text_with_page_info(pdf_file_path)
    # print(full_document_text[:2000]) # 추출된 텍스트 확인

    # 2. 초기 텍스트 분할
    print("초기 텍스트 분할 중...")
    initial_chunks = initial_text_split(full_document_text, chunk_size=3000, chunk_overlap=300) 
    # chunk_size는 LLM의 context window를 고려하여 설정 (gpt-4o는 128K 토큰)
    # 한글은 영문보다 토큰이 많이 소요되므로 실제 토큰 수를 고려해야 함
    # 3000 글자 = 대략 1000~1500 토큰 예상
    print(f"총 {len(initial_chunks)}개의 초기 청크가 생성되었습니다.")
    
    # 3. LLM을 이용한 의미적 청킹 (요구사항 추출)
    print("\nLLM을 이용한 의미적 청킹 (요구사항 추출) 시작...")
    extracted_requirements = llm_based_semantic_chunking(initial_chunks, llm_model="gpt-4o") # gpt-4o 또는 gpt-3.5-turbo 선택
    
    print(f"\n총 {len(extracted_requirements)}개의 요구사항이 추출되었습니다.")
    print("\n--- 추출된 요구사항 미리보기 (상위 5개) ---")
    for i, req in enumerate(extracted_requirements[:5]):
        print(f"ID: {req.get('id', 'N/A')}")
        print(f"Type: {req.get('type', 'N/A')}")
        print(f"Description: {req.get('description', 'N/A')}")
        print(f"Source Pages: {req.get('source_pages', 'N/A')}")
        print(f"Raw Text Snippet: {req.get('raw_text_snippet', 'N/A')[:100]}...") # 원본 스니펫 일부
        print("-" * 50)

TypeError: OpenAI.__init__() takes 1 positional argument but 2 were given